# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#507 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846835655442                   -0.70    4.5
  2   -7.852321670882       -2.26       -1.53    1.0
  3   -7.852646193085       -3.49       -2.52    3.0
  4   -7.852646676685       -6.32       -3.34    2.2
  5   -7.852646685752       -8.04       -4.76    1.2
  6   -7.852646686721       -9.01       -5.15    3.5
  7   -7.852646686729      -11.06       -5.98    1.2
  8   -7.852646686730      -12.33       -7.17    2.8
  9   -7.852646686730      -14.45       -7.40    1.8
 10   -7.852646686730   +  -14.45       -9.21    2.5
 11   -7.852646686730      -15.05       -9.39    2.2
 12   -7.852646686730   +    -Inf      -10.00    2.0
 13   -7.852646686730      -15.05      -11.56    2.2
 14   -7.852646686730   +    -Inf      -12.24    3.2


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846877279269                   -0.70           4.0
  2   -7.852525740014       -2.25       -1.62   0.80    5.0
  3   -7.852610032199       -4.07       -2.78   0.80    5.5
  4   -7.852646567649       -4.44       -3.57   0.80    7.2
  5   -7.852646683526       -6.94       -4.55   0.80    9.0
  6   -7.852646686679       -8.50       -5.09   0.80   10.2
  7   -7.852646686724      -10.35       -6.11   0.80   11.5
  8   -7.852646686730      -11.27       -7.15   0.80   13.0
  9   -7.852646686730      -13.25       -7.78   0.80   15.5
 10   -7.852646686730   +  -14.57       -8.77   0.80   16.8
 11   -7.852646686730   +  -14.75       -9.39   0.80   18.2
 12   -7.852646686730      -14.45      -10.54   0.80   19.2
 13   -7.852646686730   +  -14.45      -11.27   0.80   21.0
 14   -7.852646686730      -15.05      -11.93   0.80   23.0
 15   -7.852646686730   +  -14.75      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.428035e+01     3.734378e+00
 * time: 0.41901397705078125
     1     1.159247e+00     2.014452e+00
 * time: 0.6221389770507812
     2    -1.977490e+00     2.256236e+00
 * time: 0.6449809074401855
     3    -3.449049e+00     2.165480e+00
 * time: 0.6777980327606201
     4    -4.847891e+00     2.022690e+00
 * time: 0.7105350494384766
     5    -6.453600e+00     1.245306e+00
 * time: 0.743229866027832
     6    -7.288475e+00     7.952793e-01
 * time: 0.7758479118347168
     7    -7.570757e+00     8.135296e-01
 * time: 0.7985379695892334
     8    -7.674034e+00     5.839989e-01
 * time: 0.8215138912200928
     9    -7.746315e+00     4.083576e-01
 * time: 0.9098899364471436
    10    -7.790537e+00     8.544091e-02
 * time: 0.932826042175293
    11    -7.825960e+00     1.048041e-01
 * time: 0.955610990524292
    12    -7.842931e+00     5.190959e-02
 * time: 0.9782159328460693
    13    -7.847015e+00     4.200789e-02
 * time: 1.000836849212

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846654207459                   -0.70    4.2
  2   -7.852296185309       -2.25       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686622                   -2.54
  2   -7.852646686730       -9.97       -5.56
  3   -7.852646686730   +  -14.27      -11.40
  4   -7.852646686730      -14.75      -15.22
|ρ_newton - ρ_scf|  = 1.1199832448357651e-13
|ρ_newton - ρ_scfv| = 8.174284293162896e-14
|ρ_newton - ρ_dm|   = 5.951478705384654e-10
